In [54]:
import zarr
from zarr import Array, Group
from __future__ import annotations
import numpy as np


store = zarr.storage.ZipStore(
    "activations-gemma2-2b-the-stack-json-100k/activations_part_0000.zarr.zip",
    read_only=True,
)
z = zarr.open(store, mode="r")

In [55]:
z.tree()

/
├── activations
│   ├── layer_0 (80, 256, 2304) float16
│   ├── layer_1 (80, 256, 2304) float16
│   ├── layer_10 (80, 256, 2304) float16
│   ├── layer_11 (80, 256, 2304) float16
│   ├── layer_12 (80, 256, 2304) float16
│   ├── layer_13 (80, 256, 2304) float16
│   ├── layer_14 (80, 256, 2304) float16
│   ├── layer_15 (80, 256, 2304) float16
│   ├── layer_16 (80, 256, 2304) float16
│   ├── layer_17 (80, 256, 2304) float16
│   ├── layer_18 (80, 256, 2304) float16
│   ├── layer_19 (80, 256, 2304) float16
│   ├── layer_2 (80, 256, 2304) float16
│   ├── layer_20 (80, 256, 2304) float16
│   ├── layer_21 (80, 256, 2304) float16
│   ├── layer_22 (80, 256, 2304) float16
│   ├── layer_23 (80, 256, 2304) float16
│   ├── layer_24 (80, 256, 2304) float16
│   ├── layer_25 (80, 256, 2304) float16
│   ├── layer_26 (80, 256, 2304) float16
│   ├── layer_3 (80, 256, 2304) float16
│   ├── layer_4 (80, 256, 2304) float16
│   ├── layer_5 (80, 256, 2304) float16
│   ├── layer_6 (80, 256, 2304) float16
│   ├── layer_7 (80, 256, 2304) float16
│   ├── layer_8 (80, 256, 2304) float16
│   └── layer_9 (80, 256, 2304) float16
├── attention_mask (80, 256) int32
└── input_ids (80, 256) int32

A single zarr archive is stored stored as a `Group` with three sub-elements -- `activations`, `attention_mask` and `input_ids`. 
- The group `activations` consist of `n_layers` 3D sub-arrays of shape `(batch_size, sequence_length, hidden_dimension)`.
- The data inside the `attention_mask` is particularly important because it determines the actual length of each of the sequences. Since some of them may be shorter than max_sequence_length, the useless pad tokens need to be removed before processing.
- `input_ids` contains the actual input ids used.


In [56]:
def get_seq_length(z: Group, sample_id: int) -> int:
    attention_mask = z["attention_mask"][sample_id]
    return attention_mask.sum().item()

In [57]:
def get_one_latent(
    z: Group,
    sample_id: int,
    layer_id: int,
    position: int,
) -> np.ndarray:
    """Retrieve the latent activation for a specific batch, sample, layer, and position."""  # noqa: E501
    assert position < get_seq_length(z, sample_id), (  # noqa: S101
        f"Position out of bounds. Max position for sample {sample_id} is {get_seq_length(z, sample_id) - 1}."  # noqa: E501
    )

    return z["activations"][f"layer_{layer_id}"][sample_id, position]


In [58]:
get_one_latent(z, 1, 0, 0)

array([ 0.006042, -0.2812  ,  1.086   , ...,  0.703   , -0.3828  ,
       -0.59    ], dtype=float16)

In [59]:
z["activations"]["layer_0"][
    :,
    0,
]  # Accessing the first layer's activations for all samples at position 0

array([[ 0.006042, -0.2812  ,  1.086   , ...,  0.703   , -0.3828  ,
        -0.59    ],
       [ 0.006042, -0.2812  ,  1.086   , ...,  0.703   , -0.3828  ,
        -0.59    ],
       [ 0.006042, -0.2812  ,  1.086   , ...,  0.703   , -0.3828  ,
        -0.59    ],
       ...,
       [ 0.006042, -0.2812  ,  1.086   , ...,  0.703   , -0.3828  ,
        -0.59    ],
       [ 0.006042, -0.2812  ,  1.086   , ...,  0.703   , -0.3828  ,
        -0.59    ],
       [ 0.006042, -0.2812  ,  1.086   , ...,  0.703   , -0.3828  ,
        -0.59    ]], dtype=float16)